In [1]:
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('OnlineRetail.csv',encoding = "ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

#### Preprocessing

In [4]:
df.drop(['StockCode', 'Description', 'InvoiceNo'], axis=1, inplace=True)
df = df.loc[df['Country'] == 'United Kingdom']

In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceYearMonth'] = df['InvoiceDate'].dt.year * 100 + df['InvoiceDate'].dt.month
df['InvoiceDate'] = df['InvoiceDate'].dt.date
df['Revenue'] = df['UnitPrice'] * df['Quantity']
df

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
0,6,2010-12-01,2.55,17850.0,United Kingdom,201012,15.30
1,6,2010-12-01,3.39,17850.0,United Kingdom,201012,20.34
2,8,2010-12-01,2.75,17850.0,United Kingdom,201012,22.00
3,6,2010-12-01,3.39,17850.0,United Kingdom,201012,20.34
4,6,2010-12-01,3.39,17850.0,United Kingdom,201012,20.34
...,...,...,...,...,...,...,...
541889,12,2011-12-09,1.95,15804.0,United Kingdom,201112,23.40
541890,8,2011-12-09,2.95,13113.0,United Kingdom,201112,23.60
541891,24,2011-12-09,1.25,13113.0,United Kingdom,201112,30.00
541892,24,2011-12-09,8.95,13113.0,United Kingdom,201112,214.80


In [6]:
def sort_cluster(df, group_field, sort_field, ranked_field, by_ascending=True):
    df_star = df[[group_field,sort_field]].groupby(group_field).mean()
    df_star.sort_values(by=sort_field,ascending=by_ascending, inplace=True)
    df_star = df_star.reset_index()
    df_star[ranked_field] = df_star.index+1
    df_star.drop([sort_field], axis=1,inplace=True)
    return df_star

In [7]:
df_6m = df[(df['InvoiceYearMonth'] >= 201104) & (df['InvoiceYearMonth'] <= 201109)]
df_3m = df[(df['InvoiceYearMonth'] >= 201110) & (df['InvoiceYearMonth'] <= 201112)]
df_6m

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
142083,10,2011-04-01,1.65,18161.0,United Kingdom,201104,16.50
142084,12,2011-04-01,1.65,18161.0,United Kingdom,201104,19.80
142085,12,2011-04-01,0.85,18161.0,United Kingdom,201104,10.20
142086,2,2011-04-01,9.95,18161.0,United Kingdom,201104,19.90
142087,6,2011-04-01,2.55,18161.0,United Kingdom,201104,15.30
...,...,...,...,...,...,...,...
370926,1,2011-09-30,8.29,NaN,United Kingdom,201109,8.29
370927,1,2011-09-30,5.79,NaN,United Kingdom,201109,5.79
370928,2,2011-09-30,0.83,NaN,United Kingdom,201109,1.66
370929,2,2011-09-30,0.83,NaN,United Kingdom,201109,1.66


In [8]:
nearest_day_in_6m = df_6m[['InvoiceDate', 'CustomerID']].groupby('CustomerID').max().reset_index()
nearest_day_in_6m = nearest_day_in_6m.rename(columns={'InvoiceDate': 'NearestDay'})
nearest_day_in_6m

,CustomerID,NearestDay
0,12747.0,2011-08-22
1,12748.0,2011-09-30
2,12749.0,2011-08-18
3,12820.0,2011-09-26
4,12821.0,2011-05-09
...,...,...
2679,18278.0,2011-09-27
2680,18281.0,2011-06-12
2681,18282.0,2011-08-09
2682,18283.0,2011-09-05


In [9]:
nearest_day_in_3m = df_3m[['InvoiceDate', 'CustomerID']].groupby('CustomerID').min().reset_index()
nearest_day_in_3m = nearest_day_in_3m.rename(columns={'InvoiceDate': 'NearestComeback'})
nearest_day_in_3m

,CustomerID,NearestComeback
0,12747.0,2011-10-04
1,12748.0,2011-10-05
2,12749.0,2011-11-17
3,12820.0,2011-10-26
4,12824.0,2011-10-11
...,...,...
2350,18276.0,2011-10-27
2351,18277.0,2011-10-12
2352,18282.0,2011-12-02
2353,18283.0,2011-10-27


In [10]:
comeback_after_days = pd.merge(nearest_day_in_6m, nearest_day_in_3m, on='CustomerID', how='left')
comeback_after_days.loc[comeback_after_days['NearestComeback'].isnull(), ['NearestComeback']] =  comeback_after_days['NearestDay'] + timedelta(days=100) 
comeback_after_days

,CustomerID,NearestDay,NearestComeback
0,12747.0,2011-08-22,2011-10-04
1,12748.0,2011-09-30,2011-10-05
2,12749.0,2011-08-18,2011-11-17
3,12820.0,2011-09-26,2011-10-26
4,12821.0,2011-05-09,2011-08-17
...,...,...,...
2679,18278.0,2011-09-27,2012-01-05
2680,18281.0,2011-06-12,2011-09-20
2681,18282.0,2011-08-09,2011-12-02
2682,18283.0,2011-09-05,2011-10-27


In [11]:
comeback_after_days['Duration Comeback'] = (comeback_after_days['NearestComeback'] - comeback_after_days['NearestDay'])
comeback_after_days

,CustomerID,NearestDay,NearestComeback,Duration Comeback
0,12747.0,2011-08-22,2011-10-04,43 days
1,12748.0,2011-09-30,2011-10-05,5 days
2,12749.0,2011-08-18,2011-11-17,91 days
3,12820.0,2011-09-26,2011-10-26,30 days
4,12821.0,2011-05-09,2011-08-17,100 days
...,...,...,...,...
2679,18278.0,2011-09-27,2012-01-05,100 days
2680,18281.0,2011-06-12,2011-09-20,100 days
2681,18282.0,2011-08-09,2011-12-02,115 days
2682,18283.0,2011-09-05,2011-10-27,52 days


In [12]:
new_df = pd.merge(df, comeback_after_days, on='CustomerID')
new_df

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue,NearestDay,NearestComeback,Duration Comeback
0,32,2010-12-01,1.69,13047.0,United Kingdom,201012,54.08,2011-08-18,2011-10-14,57 days
1,6,2010-12-01,2.10,13047.0,United Kingdom,201012,12.60,2011-08-18,2011-10-14,57 days
2,6,2010-12-01,2.10,13047.0,United Kingdom,201012,12.60,2011-08-18,2011-10-14,57 days
3,8,2010-12-01,3.75,13047.0,United Kingdom,201012,30.00,2011-08-18,2011-10-14,57 days
4,6,2010-12-01,1.65,13047.0,United Kingdom,201012,9.90,2011-08-18,2011-10-14,57 days
...,...,...,...,...,...,...,...,...,...,...
313224,4,2011-09-30,4.25,16613.0,United Kingdom,201109,17.00,2011-09-30,2012-01-08,100 days
313225,4,2011-09-30,4.25,16613.0,United Kingdom,201109,17.00,2011-09-30,2012-01-08,100 days
313226,12,2011-09-30,2.10,16613.0,United Kingdom,201109,25.20,2011-09-30,2012-01-08,100 days
313227,12,2011-09-30,2.10,16613.0,United Kingdom,201109,25.20,2011-09-30,2012-01-08,100 days


In [13]:
def feature_engineering(df):
#   cluster model
    km = KMeans(n_clusters=5)
    
    
#   Processing Recency
    # max_date
    max_date = df['InvoiceDate'].max()
    df_recency = df[['CustomerID', 'InvoiceDate']].groupby('CustomerID').max()
    df_recency['RecencyDay'] = (max_date - df_recency)['InvoiceDate'].dt.days
    df_recency.drop('InvoiceDate', axis=1, inplace=True)
    df_recency = df_recency.reset_index()
    X = df_recency['RecencyDay'].to_numpy().reshape(-1,1)
    km.fit(X)
    df_cluster = pd.DataFrame({'ClusterRecency':km.labels_})
    df_recency_merge = df_recency.join(df_cluster)
    df_star_rec = sort_cluster(df_recency_merge,'ClusterRecency','RecencyDay', 'RecencyRanked', False)
    df_star_rec = pd.merge(df_recency_merge, df_star_rec, on='ClusterRecency')

#   Processing Frequency
    df_frequency = df[['CustomerID', 'InvoiceDate']].groupby('CustomerID').count()
    df_frequency = df_frequency.reset_index()
    df_frequency.columns = ['CustomerID', 'FrequencyCount']
    X = df_frequency['FrequencyCount'].to_numpy().reshape(-1,1)
    km.fit(X)
    df_cluster = pd.DataFrame({'ClusterFrequency':km.labels_})
    df_frequency_merge = df_frequency.join(df_cluster)
    df_star_fre = sort_cluster(df_frequency_merge, 'ClusterFrequency','FrequencyCount', 'FreRanked', True)
    df_star_fre = pd.merge(df_frequency_merge, df_star_fre, on='ClusterFrequency')

#   Processing Revenue
    df_revenue = df[['CustomerID', 'Revenue']].groupby('CustomerID').sum()
    df_revenue = df_revenue.reset_index()
    df_revenue.columns = ['CustomerID', 'Revenue']
    X = df_revenue['Revenue'].to_numpy().reshape(-1,1)
    km.fit(X)
    df_cluster = pd.DataFrame({'ClusterRevenue':km.labels_})
    df_revenue_merge = df_revenue.join(df_cluster)
    df_star_rev = sort_cluster(df_revenue_merge, 'ClusterRevenue','Revenue', 'RevRanked', True)
    df_star_rev = pd.merge(df_revenue_merge, df_star_rev, on='ClusterRevenue')

    overall_score = pd.merge(pd.merge(df_star_rev, df_star_fre, on='CustomerID'),df_star_rec, on='CustomerID')
    return overall_score

In [14]:
df_9m = df[df['InvoiceYearMonth'] >= 201104]
df_9m.drop_duplicates(subset=['CustomerID', 'InvoiceYearMonth'], inplace=True)
df_9m.sort_values(by=['CustomerID', 'InvoiceYearMonth'], inplace=True)
df_9m

<ipython-input-14-7b6a97817191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9m.drop_duplicates(subset=['CustomerID', 'InvoiceYearMonth'], inplace=True)
<ipython-input-14-7b6a97817191>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9m.sort_values(by=['CustomerID', 'InvoiceYearMonth'], inplace=True)


,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
175871,6,2011-05-05,2.95,12747.0,United Kingdom,201105,17.70
241991,3,2011-06-28,9.95,12747.0,United Kingdom,201106,29.85
308314,6,2011-08-22,5.95,12747.0,United Kingdom,201108,35.70
374656,10,2011-10-04,2.08,12747.0,United Kingdom,201110,20.80
478688,12,2011-11-17,6.75,12747.0,United Kingdom,201111,81.00
...,...,...,...,...,...,...,...
285724,-6,2011-08-01,0.00,NaN,United Kingdom,201108,-0.00
320707,1,2011-09-01,0.00,NaN,United Kingdom,201109,0.00
372324,2,2011-10-02,3.75,NaN,United Kingdom,201110,7.50
431798,10,2011-11-01,0.00,NaN,United Kingdom,201111,0.00


In [15]:
df_9m = df_9m[['CustomerID', 'InvoiceDate']].groupby('CustomerID').agg(['min', 'max', 'count'])  # agg de chay 1 luc 3 cai
df_9m = df_9m.reset_index()
df_9m

CustomerID InvoiceDate                  
                        min         max count
0       12747.0  2011-05-05  2011-12-07     6
1       12748.0  2011-04-01  2011-12-01     9
2       12749.0  2011-05-10  2011-12-06     4
3       12820.0  2011-09-26  2011-12-06     3
4       12821.0  2011-05-09  2011-05-09     1
...         ...         ...         ...   ...
3504    18278.0  2011-09-27  2011-09-27     1
3505    18281.0  2011-06-12  2011-06-12     1
3506    18282.0  2011-08-05  2011-12-02     2
3507    18283.0  2011-04-21  2011-12-06     8
3508    18287.0  2011-05-22  2011-10-12     2

[3509 rows x 4 columns]

In [16]:
df_9m['AvgDuration'] = (df_9m['InvoiceDate']['max'] - df_9m['InvoiceDate']['min']).dt.days / (df_9m['InvoiceDate']['count'] - 1)


In [17]:
# df_9m = df_9m.droplevel(0, axis=1)

In [18]:
df_9m = df_9m[['CustomerID', 'AvgDuration']]
df_9m = df_9m.droplevel(1, axis=1)
df_9m

,CustomerID,AvgDuration
0,12747.0,43.200000
1,12748.0,30.500000
2,12749.0,70.000000
3,12820.0,35.500000
4,12821.0,NaN
...,...,...
3504,18278.0,NaN
3505,18281.0,NaN
3506,18282.0,119.000000
3507,18283.0,32.714286


In [19]:
comeback_after_days.drop(['NearestDay', 'NearestComeback'], axis=1, inplace=True)

In [20]:
df_merge_4label = pd.merge(df_9m, comeback_after_days, on='CustomerID', how='left')